In [1]:
## Getting to Main directory
import os
os.chdir("../")

In [2]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv



e:\projects\Reseach Project - RAG\Langchain_QA\venv_RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
gemini_api_key=os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))



### Data ingestion - Taking pdf documents and Cleaning and Transforming  Data into vector index

In [4]:
from langchain_community.document_loaders import DirectoryLoader

In [75]:
loader = DirectoryLoader('Data2', glob="**/*.pdf")

In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Data/[Data] Think Like a Data Scientist (2017).pdf")
pages = loader.load_and_split()

incorrect startxref pointer(1)


In [9]:
# docs = loader.load()

In [10]:
# pages

In [11]:
len(pages)

325

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
# text_chunks = text_splitter.split_documents(docs)
text_chunks = text_splitter.split_documents(pages)

In [13]:
len(text_chunks)

1077

In [14]:
# Clean up our Documents' content
import re

def clean_up_text(content: str) -> str:
    """
    Remove unwanted characters and patterns in text input.

    :param content: Text input.
    
    :return: Cleaned version of original text input.
    """

    # Fix hyphenated words broken by newline
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

    # Remove specific unwanted patterns and characters
    unwanted_patterns = [
        "\\n", "  —", "——————————", "—————————", "—————",
        r'\\u[\dA-Fa-f]{4}', r'\uf075', r'\uf0b7'
    ]
    for pattern in unwanted_patterns:
        content = re.sub(pattern, "", content)

    # Fix improperly spaced hyphenated words and normalize whitespace
    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content

# Call function
cleaned_docs = []
for d in text_chunks: 
    cleaned_text = clean_up_text(d.page_content)
    d.page_content = cleaned_text
    cleaned_docs.append(d)

In [ ]:
clean_up_text

In [15]:
text_chunks[2].page_content

'Think Likea Data ScientistTACKLE THE DATA SCIENCE PROCESS STEP-BY-STEPBRIAN GODSEYMANNINGSHELTER ISLAND'

In [66]:
cleaned_docs[0].metadata

{'source': 'Data2\\[Data] Think Like a Data Scientist (2017).pdf', 'page': 0}

### Configuring Gemini model and GeminiEmbedding

In [67]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [68]:
genai.configure(api_key=gemini_api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# Setting Tempreture to 0.3 for getting low risk results

model = ChatGoogleGenerativeAI(model="gemini-1.0-pro-latest",api_key=gemini_api_key,temperature=0.3)

### Using FAISS as our vector database taking the index to save them locally

In [44]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

In [69]:
# for 331 page pdf
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

### Applying proper prompts to get best output

In [45]:
prompt_template = """
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
Context:\n {context}?\n
Question: \n{question}\n

Answer:
"""

# model = ChatGoogleGenerativeAI(model="gemini-pro",
#                             temperature=0.3)

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

prompt
# return chain

PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in\nprovided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\nContext:\n {context}?\n\nQuestion: \n{question}\n\n\nAnswer:\n')

In [46]:
from langchain.prompts.chat import ChatPromptTemplate
qa_prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])
qa_prompt


ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in\nprovided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\nContext:\n {context}?\n\nQuestion: \n{question}\n\n\nAnswer:\n'))])

In [47]:
chain = load_qa_chain(llm=model, chain_type="stuff", verbose=True, prompt=qa_prompt)


In [51]:
user_question = "Tell me about Employability Skills Matrix "

In [52]:
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_question)


In [53]:
docs[0]

Document(metadata={'source': 'Data2\\test.pdf'}, page_content='Employability An Employability Skills Matrix at the beginning of each chapter provides students with a visual guide to features that support the development of skills employers are looking for in today’s busi-ness graduates, helping students see the relevance of the course to their career goals from the very start of class.This matrix identifies which features and end-of-chapter material will help you develop specific skills employers are looking for in job candidates.Employability Skills Matrix (ESM)Myth or Science?An Ethical ChoicePoint/ CounterpointToward a Better WorldExperiential ExerciseEthical DilemmaCase Incident✓✓✓✓✓✓Critical Thinking & Creativity✓✓ ✓ ✓ ✓✓Communication✓ ✓ ✓Collaboration✓✓ ✓Self-Management✓✓✓Social Responsibility✓ ✓✓ ✓✓ ✓✓ ✓✓Leadership✓Career Management(Employability Skills Matrix for Chapter 2)21A01_ROBB0025_19_GE_FM.indd 21 A01_ROBB0025_19_GE_FM.indd 21')

In [54]:
response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

e:\projects\Reseach Project - RAG\Langchain_QA\venv_RAG\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer


Context:
 Employability An Employability Skills Matrix at the beginning of each chapter provides students with a visual guide to features that support the development of skills employers are looking for in today’s busi-ness graduates, helping students see the relevance of the course to their career goals from the very start of class.This matrix identifies which features and end-of-chapter material will help you develop specific skills employers are looking for in job candidates.Employability Skills Matrix (ESM)Myth or Science?An Ethical ChoicePoint/ CounterpointToward a Better WorldExperiential ExerciseEthical DilemmaCase Incident✓✓✓✓✓✓Critica

In [55]:
print(response["output_text"])

The Employability Skills Matrix (ESM) is a visual guide that provides students with a roadmap to develop the skills employers are looking for in today's business graduates. It identifies which features and end-of-chapter material will help students develop specific skills employers are looking for in job candidates. The ESM includes seven employability skills: critical thinking and creativity, communication, collaboration, self-management, social responsibility, leadership, and career management. Each chapter of the text starts with an ESM that links these skills to unique features in the chapter, such as Myth or Science?, An Ethical Choice, Point/Counterpoint, Toward a Better World, Experiential Exercise, Ethical Dilemma, and the Case Incident. Within these sections, students are primed to think critically and creatively, improve their self-management, collaboration, and communication skills, and consider the ethics of particular behaviors in the workplace.


In [42]:
eval_questions = [
    "How many baby birds did the mother bird have?",
    "What happened to the mother bird during the storm?",
    "Where did the baby birds end up after the storm?",
    "What happened to one of the baby birds near a cave?",
    "Where did the other baby bird land?",
    "Who came to the forest one day?",
    "What did the king see under a tree near a cave?",
    "What did the king hear from the big, brown bird?",
    "Where did the king go after leaving the cave?",
    "Who welcomed the king at the ashram?",
    "What did the rishi say about the two birds?",
    "What did the king tell the rishi about the birds?",
    "What did the rishi say to the king about the birds?",
    "What did the rishi compare the birds' behavior to?",
    "Which moral best summarizes the story?"
]

eval_answers = [
    "The mother bird had two baby birds.",
    "The mother bird was killed by a heavy branch during the storm.",
    "The baby birds ended up separated from each other after the storm.",
    "One of the baby birds came down near a cave where a gang of robbers lived.",
    "The other baby bird landed outside a rishi’s ashram a little distance away.",
    "The king of the country came to the forest one day.",
    "The king saw a big, brown bird under a tree near a cave.",
    "The king heard the big, brown bird urging someone to take his jewels and horse quickly.",
    "After leaving the cave, the king came to a clearing that looked like an ashram.",
    "The king was welcomed at the ashram by a big, brown bird.",
    "The rishi said that one bird had made friends with robbers while the other welcomed people to the ashram.",
    "The king told the rishi the story of the two birds and their different behaviors.",
    "The rishi said that one is known by the company one keeps.",
    "The rishi compared the birds' behavior to the influence of companionship.",
    "The moral 'One is known by the company one keeps' best summarizes the story."
]


In [44]:
examples = [
    {"query": q, "ground_truth": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [45]:
examples

[{'query': 'How many baby birds did the mother bird have?',
  'ground_truth': ['The mother bird had two baby birds.']},
 {'query': 'What happened to the mother bird during the storm?',
  'ground_truth': ['The mother bird was killed by a heavy branch during the storm.']},
 {'query': 'Where did the baby birds end up after the storm?',
  'ground_truth': ['The baby birds ended up separated from each other after the storm.']},
 {'query': 'What happened to one of the baby birds near a cave?',
  'ground_truth': ['One of the baby birds came down near a cave where a gang of robbers lived.']},
 {'query': 'Where did the other baby bird land?',
  'ground_truth': ['The other baby bird landed outside a rishi’s ashram a little distance away.']},
 {'query': 'Who came to the forest one day?',
  'ground_truth': ['The king of the country came to the forest one day.']},
 {'query': 'What did the king see under a tree near a cave?',
  'ground_truth': ['The king saw a big, brown bird under a tree near a cave

In [46]:
user_question = eval_questions[5]

In [47]:
user_question

'Who came to the forest one day?'

In [48]:
new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_question)

docs

[Document(page_content='would the crocodile tell his wife?Rationalised 2023-24', metadata={'source': 'Data\\fepw105.pdf'}),
 Document(page_content='Rationalised 2023-24', metadata={'source': 'Data\\fepw106.pdf'}),
 Document(page_content='Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24', metadata={'source': 'Data\\fepw104.pdf'}),
 Document(page_content='Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry out, “Quick! Hurry up! There’s someone under the tree. Come and take his jewels and his horse. Hurry, or else he’ll slip away.” Th

In [49]:
response = chain(
        {"input_documents":docs, "question": user_question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer


Context:
 would the crocodile tell his wife?Rationalised 2023-24

Rationalised 2023-24

Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24

Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry

In [50]:
response["input_documents"][0].page_content

'would the crocodile tell his wife?Rationalised 2023-24'

In [51]:
print(response["output_text"])

## Who Came to the Forest? 

**The king of the country** came to the forest one day to hunt. 



In [52]:

import nest_asyncio

nest_asyncio.apply()


In [53]:
response

{'input_documents': [Document(page_content='would the crocodile tell his wife?Rationalised 2023-24', metadata={'source': 'Data\\fepw105.pdf'}),
  Document(page_content='Rationalised 2023-24', metadata={'source': 'Data\\fepw106.pdf'}),
  Document(page_content='Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24', metadata={'source': 'Data\\fepw104.pdf'}),
  Document(page_content='Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry out, “Quick! Hurry up! There’s someone under the tree. Come and take his jewels and his horse. Hurry, or el

## RAG model Evaluation using RAGAS; Currently RAGAS is not supporting Gemini, but I do hope it would be available in future. Regardless of that, Integrating Open AI with RAGAS won't be a problem if we already have access to it's api key

In [55]:
from ragas.langchain.evalchain  import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# create evaluation chains
# faithfulness_chain = RagasEvaluatorChain(metric=faithfulness)
# answer_rel_chain = RagasEvaluatorChain(metric=answer_relevancy)
# context_rel_chain = RagasEvaluatorChain(metric=context_precision)
# context_recall_chain = RagasEvaluatorChain(metric=context_recall)

In [56]:
import pandas as pd

In [57]:
# Create a DataFrame
test_df = pd.DataFrame({
    'question': eval_questions,
    'ground_truth': eval_answers
})

# Print the DataFrame
print(test_df)

                                             question  \
0       How many baby birds did the mother bird have?   
1   What happened to the mother bird during the st...   
2    Where did the baby birds end up after the storm?   
3   What happened to one of the baby birds near a ...   
4                 Where did the other baby bird land?   
5                     Who came to the forest one day?   
6     What did the king see under a tree near a cave?   
7    What did the king hear from the big, brown bird?   
8       Where did the king go after leaving the cave?   
9                Who welcomed the king at the ashram?   
10        What did the rishi say about the two birds?   
11  What did the king tell the rishi about the birds?   
12  What did the rishi say to the king about the b...   
13  What did the rishi compare the birds' behavior...   
14             Which moral best summarizes the story?   

                                         ground_truth  
0                 The mother bi

In [58]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [59]:
test_questions = test_questions[:2]
test_groundtruths = test_groundtruths[:2]

In [60]:
test_questions

['How many baby birds did the mother bird have?',
 'What happened to the mother bird during the storm?']

In [61]:
answers = []
contexts = []

for question in test_questions:
      new_db = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
      docs = new_db.similarity_search(user_question)

      response = chain(
            {"input_documents":docs, "question": question})
      # response = retrieval_augmented_qa_chain.invoke({"question" : question})
      answers.append(response["output_text"])
      contexts.append([context.page_content for context in response["input_documents"]])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer


Context:
 would the crocodile tell his wife?Rationalised 2023-24

Rationalised 2023-24

Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24

Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood near a cave. Suddenly he heard a voice cry

In [62]:
test_groundtruths

['The mother bird had two baby birds.',
 'The mother bird was killed by a heavy branch during the storm.']

In [67]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [80]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truths" : test_groundtruths
})

In [88]:
from datasets import Dataset, Features, Sequence, Value

# Convert each ground truth string into a list containing that string
test_groundtruths_list = [[gt] for gt in test_groundtruths]

# Define the features with the correct type for "ground_truths"
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "contexts": Value("string"),
    "ground_truths": Sequence(Sequence(Value("string")))
})

# Create the dataset
response_dataset = Dataset.from_dict({
    "question": test_questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": test_groundtruths_list
})

# Now, you should be able to use the dataset without encountering the ValueError


In [89]:
response_dataset[0]

{'question': 'How many baby birds did the mother bird have?',
 'answer': '## Analyzing the Context for Baby Bird Count:\n\nThe provided text describes a story involving a king, a deer, and birds but **does not specify the number of baby birds the mother bird had**. Therefore, based on the context provided, the answer is: \n\n**Answer is not available in the context** \n',
 'contexts': ['would the crocodile tell his wife?Rationalised 2023-24',
  'Rationalised 2023-24',
  'Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24',
  'Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood

In [90]:
response_dataset[0]

{'question': 'How many baby birds did the mother bird have?',
 'answer': '## Analyzing the Context for Baby Bird Count:\n\nThe provided text describes a story involving a king, a deer, and birds but **does not specify the number of baby birds the mother bird had**. Therefore, based on the context provided, the answer is: \n\n**Answer is not available in the context** \n',
 'contexts': ['would the crocodile tell his wife?Rationalised 2023-24',
  'Rationalised 2023-24',
  'Are you interested in music? Do you like classical music? Name a few distinguished Indian musicians.Rationalised 2023-24',
  'Days passed and the baby birds became big birds. One day, the king of the country came to the forest to hunt. He saw a deer and rode after it. It ran deep into the forest followed by the king. Soon the king lost his way and didn’t know where he was.He rode on for a long time till he came to the other side of the forest. Very tired by now, he got off his horse and sat down under a tree that stood

In [ ]:

advanced_retrieval_results = evaluate(response_dataset, metrics)